In [ ]:
""" 同じ条件の実行フォルダをまとめる. """
import os
import shutil
from glob import glob
from pathlib import Path
import numpy as np
from collections import defaultdict
from itertools import combinations
dset_dict = {
    'office': ["amazon", "dslr", "webcam"],
    'OfficeHome': ["Art", "Clipart", "Product", "RealWorld"],
    'DomainNet': ['clipart_infograph', 'clipart_painting', 'clipart_quickdraw', 'clipart_real', 'clipart_sketch', 'infograph_painting', 'infograph_quickdraw', 'infograph_real', 'infograph_sketch', 'painting_quickdraw', 'painting_real', 'painting_sketch', 'quickdraw_real', 'quickdraw_sketch', 'real_sketch'],
}
dset_combs = {}
for parent, dl in dset_dict.items():
    dset_combs[parent] = [f'{a[0][0]}{a[1][0]}' for a in combinations(dl, 2)]
    dset_combs[parent] += [f'{a[1][0]}{a[0][0]}' for a in combinations(dl, 2)]
group_list = defaultdict(list)
parent = 'office'
folder = f'log/{parent}'
dirs = glob(f'{folder}/*/*')
for dir in dirs:
    time, gpu, dset, group = Path(dir).name.split('--')
    if dset not in dset_combs[parent]:
        continue
    group_list[group].append(dir)

for group, dirs_list in group_list.items():
    for dir in dirs_list:
        to_dir = Path(dir).parent / group
        os.makedirs(to_dir, exist_ok=True)
        shutil.move(dir, to_dir)
        

In [ ]:
import os
import shutil
from glob import glob
from pathlib import Path
import numpy as np
from collections import defaultdict

parent = 'office'
print(f'-----  {parent}  -----')
logs = glob(f'log/{parent}/*/*/*/log.txt')
group_results = defaultdict(dict)
for log in logs:
    with open(log, 'r') as f:
        lines = f.readlines()
    acc = np.max([float(line.split(': ')[-1][:-2]) for line in lines if 'acc_test' in line])
    n = Path(log).parent.name.split('--')[-1]
    dset = Path(log).parent.name.split('--')[2]

    group_results[n][dset] = acc

rslt_list = []
for group, dset_acc in group_results.items():
    mean = np.mean([acc for acc in dset_acc.values()])
    sorte_dset_acc = dict(sorted(dset_acc.items(), key=lambda x: x[0]))
    rslt = f'\tmean: {mean:.2f}%\t' + '\t'.join([f'{dset}: {acc:.2f}%' for dset, acc in sorte_dset_acc.items()])
    rslt_list.append((mean, group, rslt))
for mean, group, rslt in sorted(rslt_list, key=lambda x: x[0], reverse=True):
    print(group)
    print(rslt)
print('\n\n')
